In [ ]:
# Importing all the necessary libraries:-
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\JAI RAHUL\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\JAI RAHUL\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\JAI RAHUL\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: U

In [2]:
#Load the model:-
model=load_model("churn_model.h5")

#Load all the pickle files:-
with open("le_gender.pkl","rb") as file:
    le_gender=pickle.load(file)

with open("scaler.pkl","rb") as file:
    scaler=pickle.load(file)

with open("scale.pkl","rb") as file:
    scale=pickle.load(file)

In [3]:
# Input-Features for predictions:-
input_data={
    'CreditScore': 500,
    'Geography': 'Spain',
    'Gender': 'Male',
    'Age': 45,
    'Tenure': 4,
    'Balance': 140000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 90000
}

In [4]:
df_in=pd.DataFrame([input_data])
df_in.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,Spain,Male,45,4,140000,3,1,1,90000


In [5]:
# One-Hot Encoding(OHE) on Feature-"Geography":-
ohe_geo=scaler.transform([[input_data["Geography"]]]).toarray()
ohe_geo_df=pd.DataFrame(ohe_geo,columns=scaler.get_feature_names_out(["Geography"]))
ohe_geo_df

c:\Users\JAI RAHUL\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [6]:
df_in["Gender"]=le_gender.transform(df_in["Gender"])
df_in

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,500,Spain,1,45,4,140000,3,1,1,90000


In [7]:
# Concatenation of necessary features:-
df_in=df_in.drop("Geography",axis=1)
df_in=pd.concat([df_in,ohe_geo_df],axis=1)
df_in

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,500,1,45,4,140000,3,1,1,90000,0.0,0.0,1.0


In [8]:
# Scaling the features:-
scaled_df=scale.transform(df_in)
scaled_df

array([[-1.57375827,  0.91324755,  0.58015577, -0.3483691 ,  1.02306797,
         2.53355998,  0.64920267,  0.97481699, -0.1813654 , -0.99850112,
        -0.57946723,  1.73494238]])

In [9]:
## Predict of churn
prediction=model.predict(scaled_df)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


array([[0.995562]], dtype=float32)

In [10]:
if prediction > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is likely to churn.
